In [21]:
import yfinance as yf
import pandas as pd
import os
import datetime
import pickle

In [ ]:
# Some important functions
def save_obj(obj, name):
    with open('data/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
def getTickerDat(ticks):
tickerDat = {}
for t in ticks:
    print('{}% done'.format(round(100*(len(tickerDat)+1)/len(ticks)),6))
    stockDat = yf.download(t, 
                           start=insiderDat.TradeDate.min().strftime("%Y-%m-%d"), 
                           end=insiderDat.TradeDate.max().strftime("%Y-%m-%d"), 
                           progress=False)
    tickerDat[t] = stockDat
return tickerDat

In [18]:
insiderDat = pd.read_csv('data/openinsider_June.csv')

In [25]:
insiderDat = insiderDat.rename(columns={insiderDat.columns[1]: "FilingDate", 
                                        insiderDat.columns[2]: "TradeDate", 
                                        insiderDat.columns[4]: "CompanyName", 
                                        insiderDat.columns[5]: "InsiderName", 
                                        insiderDat.columns[7]: "TradeType",
                                        insiderDat.columns[11]: "DeltaOwn"},
                              errors="raise")
insiderDat.FilingDate = [datetime.datetime.strptime(r, '%m/%d/%y %H:%M') for r in insiderDat.FilingDate]
insiderDat.TradeDate = [datetime.datetime.strptime(r, '%m/%d/%y') for r in insiderDat.TradeDate]

insiderDat.head()

,X,FilingDate,TradeDate,Ticker,CompanyName,InsiderName,Title,TradeType,Price,Qty,Owned,DeltaOwn,Value,1d,1w,1m,6m
0,NaN,2021-06-11 18:00:00,2021-06-11,IBP,"Installed Building Products, Inc.",Thomas Michael H,Dir,S - Sale,$116.85,"-2,100","10,093",-17%,"($245,378)",NaN,NaN,NaN,NaN
1,NaN,2021-06-11 19:21:00,2021-06-11,DXC,Dxc Technology Co,Fernandez Raul J,Dir,P - Purchase,$41.20,"1,820","31,305",6%,"$74,975",NaN,NaN,NaN,NaN
2,NaN,2021-06-11 16:50:00,2021-06-11,REVG,"Rev Group, Inc.",Boettinger Stephen W.,GC,P - Purchase,$15.64,"1,500","121,560",1%,"$23,460",NaN,NaN,NaN,NaN
3,NaN,2021-06-11 19:43:00,2021-06-11,AMST,Amesite Inc.,Sastry Ann Marie,CEO,P - Purchase,$3.73,"15,000","6,241,667",0%,"$55,950",NaN,NaN,NaN,NaN
4,NaN,2021-06-11 19:47:00,2021-06-11,TLYS,"Tilly'S, Inc.",Zeichner Bernard,Dir,S - Sale,$16.24,"-5,000","46,894",-10%,"($81,200)",NaN,NaN,NaN,NaN


In [26]:
allTickers = insiderDat.Ticker.unique()
len(allTickers)

913

In [29]:
#historicDat = getTickerDat(allTickers)
#save_obj(historicDat, 'historicDat')

0% done
0% done
0% done
0% done
1% done
1% done
1% done
1% done
1% done
1% done
1% done
1% done
1% done
2% done
2% done
2% done
2% done
2% done
2% done
2% done
2% done
2% done
3% done
3% done
3% done
3% done
3% done
3% done
3% done
3% done
3% done
4% done
4% done
4% done
4% done
4% done
4% done
4% done
4% done
4% done
4% done
5% done
5% done
5% done
5% done
5% done
5% done
5% done
5% done
5% done
6% done
6% done
6% done
6% done
6% done
6% done
6% done
6% done
6% done
7% done
7% done
7% done
7% done
7% done
7% done
7% done
7% done
7% done
8% done
8% done
8% done
8% done
8% done
8% done
8% done
8% done
8% done
9% done
9% done
9% done
9% done
9% done
9% done
9% done
9% done
9% done
10% done
10% done
10% done
10% done
10% done
10% done
10% done
10% done
10% done
11% done
11% done
11% done
11% done
11% done
11% done
11% done
11% done
11% done
12% done
12% done
12% done
12% done
12% done
12% done

1 Failed download:
- SVMK: No data found, symbol may be delisted
12% done
12% done
12% done
12%

89% done
89% done
89% done
90% done
90% done
90% done
90% done
90% done
90% done
90% done
90% done
90% done
91% done
91% done
91% done
91% done
91% done
91% done
91% done
91% done
91% done
92% done
92% done
92% done
92% done
92% done
92% done
92% done
92% done
92% done
93% done
93% done
93% done
93% done
93% done
93% done
93% done
93% done
93% done
94% done
94% done
94% done
94% done
94% done
94% done
94% done
94% done
94% done
95% done
95% done
95% done
95% done
95% done
95% done
95% done
95% done
95% done
96% done
96% done
96% done
96% done
96% done
96% done
96% done
96% done
96% done
96% done
97% done
97% done
97% done
97% done
97% done
97% done
97% done
97% done
97% done
98% done
98% done
98% done
98% done
98% done
98% done
98% done
98% done
98% done
99% done
99% done
99% done
99% done
99% done

1 Failed download:
- HTZGQ: No data found, symbol may be delisted
99% done
99% done
99% done
99% done
100% done
100% done
100% done
100% done
100% done


In [41]:
ghostTickers = ['SVMK', 'ISNS', 'GTXMQ', 'TCF', 'HOME', 'ARYA', 'ZANDZ', 'DILA', 'RACA', 'WPF', 'TRCH', 'USCR', 
                'KIN', 'STAY', 'HTZGQ']

insiderDat_filtered = insiderDat[insiderDat.Ticker.isin(ghostTickers)==False]
insiderDat_filtered.head()

,X,FilingDate,TradeDate,Ticker,CompanyName,InsiderName,Title,TradeType,Price,Qty,Owned,DeltaOwn,Value,1d,1w,1m,6m
0,NaN,2021-06-11 18:00:00,2021-06-11,IBP,"Installed Building Products, Inc.",Thomas Michael H,Dir,S - Sale,$116.85,"-2,100","10,093",-17%,"($245,378)",NaN,NaN,NaN,NaN
1,NaN,2021-06-11 19:21:00,2021-06-11,DXC,Dxc Technology Co,Fernandez Raul J,Dir,P - Purchase,$41.20,"1,820","31,305",6%,"$74,975",NaN,NaN,NaN,NaN
2,NaN,2021-06-11 16:50:00,2021-06-11,REVG,"Rev Group, Inc.",Boettinger Stephen W.,GC,P - Purchase,$15.64,"1,500","121,560",1%,"$23,460",NaN,NaN,NaN,NaN
3,NaN,2021-06-11 19:43:00,2021-06-11,AMST,Amesite Inc.,Sastry Ann Marie,CEO,P - Purchase,$3.73,"15,000","6,241,667",0%,"$55,950",NaN,NaN,NaN,NaN
4,NaN,2021-06-11 19:47:00,2021-06-11,TLYS,"Tilly'S, Inc.",Zeichner Bernard,Dir,S - Sale,$16.24,"-5,000","46,894",-10%,"($81,200)",NaN,NaN,NaN,NaN


In [42]:
len(insiderDat_filtered.Ticker.unique())

898

In [43]:
insiderDat_filtered.to_csv('data/insiderDat_June_clean.csv', index=False)